In [ ]:
from diffusers import StableDiffusionXLPipeline
import torch

base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = StableDiffusionXLPipeline.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

lora_model_path = "./sdxl-lora-minhwa-style" # set your LoRA model path
pipe.load_lora_weights(lora_model_path)

prompt = (
    "orean traditional minhwa painting, "
    "two person, "
    "a men wearing Korean hat, gat, blue clothes, "
    "a men wearing blue clothes, "
    "with a background of green mountain landscape."
) # set your prompt
image = pipe(prompt=prompt, num_inference_steps=30).images[0]

image.save("minhwa_test.png")